# Exploration of daytime columns
- 'PurchDate', 
- 'VehYear', 
- 'VehicleAge'

In [ ]:
## add current directory to system path for relative import of the castom pakage: avp_pckg
import sys
import os

## geting one level above current path 
path_absolute = os.path.dirname(os.getcwd()) 

if path_absolute in sys.path:
    print(f"alrady in sys.path: {path_absolute}")
else:
    sys.path.append(os.path.dirname(os.getcwd()))
    print(f"added to sys.path: {path_absolute}"  )

## for debagging     
# for p in sys.path:
#     print(p)
    
## check import from avp_pckg
from avp_pckg.DataFrame import AvPdataFrame 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


### Load original data


In [ ]:
df = pd.read_csv(path_absolute + '\\data\\DontGetKicked\\training.csv', 
                 parse_dates=['PurchDate'], 
                 index_col=0)
display(df.head(3))
#df.info()

# EDA

In [ ]:
col_year= ['IsBadBuy', 'PurchDate', 'VehYear', 'VehicleAge']
df_year = df[col_year].copy()
df_year.loc[:, 'PurchYear'] = df_year['PurchDate'].dt.year
df_year.loc[:, 'PurchMonth'] = df_year['PurchDate'].dt.month
df_year.loc[:, 'PurchDay'] = df_year['PurchDate'].dt.day
## for undestandig correlation between  'VehYear' and 'VehicleAge' columns
df_year.loc[:, 'Age2'] = df_year['PurchYear'] - df_year['VehYear']
df_year.head()


In [ ]:
mask = df_year['VehicleAge' ] != df_year['Age2']
df_wrong_year = df_year[mask]
print('number of entries in df_wrong_year: ', df_wrong_year.shape[0]) 
df_wrong_year.head(7)

In [ ]:
df_year['PurchYear'].unique()

### conclusion
- strong correlation between 'VehYear' and 'VehicleAge' is related to fact that all cars were bought within 2-years period. 
- PurchYear - 'VehYear' = 'VehicleAge' (just 7 exeptions, month of the year was not considered)
- 'VehYear' can be an additional indication for car model? month is not specified. 
- 'VehicleAge' looks to be rounded on the base of PurchMonth. 

## Total IsBadBuy frequency

In [ ]:
bad_buys = df_year['IsBadBuy'].sum()
total_buys = df_year.shape[0]
bad_buys/total_buys *100

## VehicleAge vs IsBadBuy frequency

In [ ]:
# %load_ext autoreload
# %autoreload 2

%reload_ext autoreload

from avp_pckg.DataFrame import AvPdataFrame 
avp_df = AvPdataFrame(df_year)
# 'VehicleAge'
df_tmp = avp_df.calc_frequency(col='VehYear', target='IsBadBuy').sort_index()
display(df_tmp)
df_tmp.plot(y='IsBadBuy_%', use_index=True)

In [ ]:
# 'VehicleAge'
df_tmp = avp_df.calc_frequency(col='VehicleAge', target='IsBadBuy').sort_index()
display(df_tmp)
df_tmp.plot(y='IsBadBuy_%', use_index=True)

conclusions: 
- linear correlation of IsBadBuy frequency with 'VehicleAge'
- dependence of IsBadBuy frequency on 'VehYear' looks bit more smooth => drop 'VehicleAge'

## PurchDate vs IsBadBuy frequency

In [ ]:
## 'IsBadBuy_%' , 'count'
avp_df.calc_frequency(col='PurchMonth', target='IsBadBuy').sort_values('IsBadBuy_%')

In [ ]:
avp_df.calc_frequency(col='PurchDay', target='IsBadBuy').sort_values('IsBadBuy_%')

In [ ]:
cols = ['VehYear', 'VehicleAge', 'PurchYear', 'PurchMonth', 'PurchDay']
df_impot = avp_df.cols_importance(cols=cols, target='IsBadBuy')
df_impot.head()

### Conclusion
- avearaged IsBadBuy = 12.3 +/- 2.3 % 
- there is no corelation of the IsBadBuy with day, month or year of purch => estimation of random fuctuations 
- => 5% of IsBadBay is a threshold for disctinguishing categories 